In [1]:
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm

from sklearn.externals import joblib

import pandas as pd
import numpy as np
from datetime import datetime

TRAIN_PATH = '../../res/ftr/base_data_train.csv'
EVALUATION_PATH = '../../res/ftr/base_data_evaluation.csv'

pd.set_option('display.max_columns', 30)

training_set = pd.read_csv(TRAIN_PATH)
evaluation_set = pd.read_csv(EVALUATION_PATH)

training_set.drop(columns=['Unnamed: 0'], inplace=True)
evaluation_set.drop(columns=['Unnamed: 0'], inplace=True)

/home/pablo/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def load_features(train_df, evaluation_df, features_list):
    for feature in features_list:
        test_ftr = pd.read_csv('../../res/ftr/'+feature+'_train.csv')
        evaluation_ftr = pd.read_csv('../../res/ftr/'+feature+'_evaluation.csv')
    
        train_df = train_df.merge(test_ftr, on='id', how='inner').drop(columns=['Unnamed: 0'])
        evaluation_df = evaluation_df.merge(evaluation_ftr, on='id', how='inner').drop(columns=['Unnamed: 0'])
    
    return (train_df, evaluation_df)

features_array = ['amenities', 'avenida_in_direction', 'encoded_provincia', 'encoded_tipodepropiedad',
                  'feature_hashed_ciudad', 'mean_precio_encoded_ciudad', 'mean_precio_encoded_provincia',
                  'mean_precio_encoded_tipodepropiedad', 'metros_totales_y_cubiertos_log', 'murder_rate_of_entity',
                  'provincia_borders_analysis', 'provincia_economy', 'qualificative_adjectives_in_description',
                  'surface_features', 'mean_idzona_price','security_words_in_description',
                  'metrostotales_bigger_than_metroscubiertos']
(training_set, evaluation_set) = load_features(training_set, evaluation_set, features_array)

In [7]:
training_set_X = training_set.drop(columns=['precio']).values
training_set_Y = training_set['precio'].values

lgbm_regressor = joblib.load('../../res/reg/LightGBM/lightgbm_regressor')

cb_regressor = cb.CatBoostRegressor()
cb_regressor.load_model('../../res/reg/CatBoost/catboost_regressor')

xgb_regressor = xgb.XGBRegressor()
xgb_regressor.load_model('../../res/reg/XGBoost/xgboost_regressor')

In [8]:
stackcv_regressor = StackingCVRegressor(regressors=(lgbm_regressor, cb_regressor, xgb_regressor),
                                        meta_regressor = xgb_regressor,
                                        use_features_in_secondary = True)



stackcv_regressor.fit(training_set_X, training_set_Y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').